In [1]:
# step1: 数据预处理
# customer_pur_recoder={vipno:[purTime，pluno, category:[]]}

# step2: 对于每个客户，构造FTCTree
# ftctree_dict={vipno:FTCTree}

# //实现GetCT函数
# //由几个更小的函数组成
# step3: 求unionTree（利用迭代，temp_unionTree与新客户FTCTree进行union即可）

# step4: 求AvgFreq(utree)和MaxFreq(utree)

# step5: 实现update(utree,freq)函数

# step6: 实现Dist(FTCTree,utree)函数

# //实现BIC
# step7: 实现簇的分裂算法（Kmeans）

# step8: 实现求bic函数，对于分裂前后的簇，分别求bic

## 数据预处理，得到数据：customer_pur_recoder = { vipno: \[purTime，pluno, category:\[ \] \]}

In [89]:
import pandas as pd
import json
import datetime
from IPython.display import HTML
from treelib import Node, Tree
from numpy import mean
import random
from copy import deepcopy
import math
import operator

In [3]:
df = pd.read_csv('../trade_new.csv')

# 取出所需要的信息并根据vipno排序
useful_info = df[['vipno', 'pluno', 'sldatime']].sort_values('vipno')
useful_info.reset_index(drop=True, inplace=True)
HTML(useful_info.to_html(index=False,max_rows=100))

vipno,pluno,sldatime
781924,15120000,2016-07-30 14:03:06
781924,11533012,2016-05-01 13:48:21
781924,14403083,2016-05-01 13:48:21
781924,15202012,2016-06-01 21:12:26
781924,34023002,2016-07-30 14:03:06
781924,11533012,2016-05-01 13:48:21
781924,11533012,2016-04-04 19:12:24
781924,10130009,2016-04-04 19:12:24
781924,11531020,2016-04-04 19:12:24
781924,14402009,2016-04-04 19:12:24


In [4]:
# 取出所有的vipno列表（去重）
vipno = useful_info.drop_duplicates(subset=['vipno'], keep='first', inplace=False)['vipno']
vipno

0                     781924
24               13325038116
46               13854627199
97               13864739266
136              15954611837
165              15954688237
207              15963883482
276              15963885355
312              18554652702
344              18654692914
390              18764569988
396            1590120464497
475            1590120718170
483            1590130640102
552            1590130817948
653            1590140304209
713            1590140304506
797            1590140305107
870            1590140306678
930            1590140307286
977            1590140307415
1045           1590140307767
1114           1590140307859
1144           1590140308030
1220           1590140308214
1306           1590141216259
1338           1590141414228
1360           1590142128124
1444           1590142129763
1487           1590142148962
                ...         
21089          2900002934915
21125          2900002936520
21160          2900002936940
21262         

In [5]:
customer_pur_recoder = {}
# 以每个vipno为key，值为{pluno: [], sldatime: []，category: []}
for item in vipno:
    customer_pur_recoder[item]={}
    customer_pur_recoder[item]['pluno'] = useful_info.loc[useful_info['vipno'] == item]['pluno'].tolist()
    customer_pur_recoder[item]['sldatime'] = useful_info.loc[useful_info['vipno'] == item]['sldatime'].tolist()
    customer_pur_recoder[item]['category'] = []
customer_pur_recoder

{781924: {'pluno': [15120000,
   11533012,
   14403083,
   15202012,
   34023002,
   11533012,
   11533012,
   10130009,
   11531020,
   14402009,
   11532036,
   14101028,
   14082002,
   15200007,
   11302032,
   11532011,
   14050019,
   15200007,
   15200007,
   15200001,
   14014034,
   23113024,
   10113009,
   15113000],
  'sldatime': ['2016-07-30 14:03:06',
   '2016-05-01 13:48:21',
   '2016-05-01 13:48:21',
   '2016-06-01 21:12:26',
   '2016-07-30 14:03:06',
   '2016-05-01 13:48:21',
   '2016-04-04 19:12:24',
   '2016-04-04 19:12:24',
   '2016-04-04 19:12:24',
   '2016-04-04 19:12:24',
   '2016-06-01 21:12:26',
   '2016-07-30 14:03:06',
   '2016-07-30 14:03:06',
   '2016-06-01 21:12:26',
   '2016-07-16 20:04:23',
   '2016-04-04 19:12:24',
   '2016-04-04 19:12:24',
   '2016-04-04 19:12:24',
   '2016-05-01 13:48:21',
   '2016-07-30 14:03:06',
   '2016-04-04 19:12:24',
   '2016-07-30 14:03:06',
   '2016-05-01 13:48:21',
   '2016-06-01 21:12:26'],
  'category': []},
 13325038116: 

In [6]:
# 求所有客户中最近一次的购买时间
t_df = df['sldatime'].sort_values(ascending = False)
t_df.reset_index(drop=True, inplace=True)
latest_time = datetime.datetime.strptime(t_df[0],'%Y-%m-%d %H:%M:%S')
latest_time

datetime.datetime(2016, 7, 31, 21, 14, 7)

In [7]:
# 得到category
# 遍历所有客户
for index in customer_pur_recoder:
    # 对每个客户，遍历所有购买记录
    customer_pur_recoder[index]['category'] = []
    for index2,item in enumerate(customer_pur_recoder[index]['sldatime']):
        t_category = []
        temp_time = datetime.datetime.strptime(item,'%Y-%m-%d %H:%M:%S')
        diff_days = (latest_time - temp_time).days
        t_category.append(int(customer_pur_recoder[index]['pluno'][index2]/1000000))
        if diff_days <= 120:
            t_category.append(int(customer_pur_recoder[index]['pluno'][index2]/100000))        
        if diff_days <= 60:
            t_category.append(int(customer_pur_recoder[index]['pluno'][index2]/10000))
        if diff_days <= 30:
            t_category.append(int(customer_pur_recoder[index]['pluno'][index2]/1000))
        customer_pur_recoder[index]['category'].append(t_category.copy())
customer_pur_recoder

{781924: {'pluno': [15120000,
   11533012,
   14403083,
   15202012,
   34023002,
   11533012,
   11533012,
   10130009,
   11531020,
   14402009,
   11532036,
   14101028,
   14082002,
   15200007,
   11302032,
   11532011,
   14050019,
   15200007,
   15200007,
   15200001,
   14014034,
   23113024,
   10113009,
   15113000],
  'sldatime': ['2016-07-30 14:03:06',
   '2016-05-01 13:48:21',
   '2016-05-01 13:48:21',
   '2016-06-01 21:12:26',
   '2016-07-30 14:03:06',
   '2016-05-01 13:48:21',
   '2016-04-04 19:12:24',
   '2016-04-04 19:12:24',
   '2016-04-04 19:12:24',
   '2016-04-04 19:12:24',
   '2016-06-01 21:12:26',
   '2016-07-30 14:03:06',
   '2016-07-30 14:03:06',
   '2016-06-01 21:12:26',
   '2016-07-16 20:04:23',
   '2016-04-04 19:12:24',
   '2016-04-04 19:12:24',
   '2016-04-04 19:12:24',
   '2016-05-01 13:48:21',
   '2016-07-30 14:03:06',
   '2016-04-04 19:12:24',
   '2016-07-30 14:03:06',
   '2016-05-01 13:48:21',
   '2016-06-01 21:12:26'],
  'category': [[15, 151, 1512, 15

## 对于每个客户，构造FTCTree

In [8]:
# 定义节点 node = {clifi:pluno/10**(7-d),freq:n} d为节点的深度
ftctree_dict = {}
# i = 0
for index in customer_pur_recoder:
#     if i > 0:
#         break
#     i+=1
    c_tree = Tree()
    c_tree.create_node(index,'root')
    for category in customer_pur_recoder[index]['category']:
        parent = 'root'
        for item in category:
            if c_tree.contains(item):
                c_tree[item].data += 1
            else:
                c_tree.create_node(item, item, data = 1, parent= parent)
            parent = item
    ftctree_dict[index] = c_tree
c_tree.show()
ftctree_dict
# print(c_tree.to_json(with_data=True))

6227002180921663895
├── 10
├── 11
│   └── 111
├── 14
│   └── 140
│       └── 1406
│           └── 14063
├── 15
├── 21
│   └── 210
├── 22
│   └── 221
│       └── 2210
├── 23
│   └── 231
│       └── 2313
│           └── 23132
├── 24
│   └── 241
│       └── 2410
│           └── 24100
├── 25
│   └── 251
│       └── 2510
├── 27
│   ├── 270
│   │   └── 2700
│   └── 274
│       └── 2740
└── 30
    └── 303
        └── 3038



{781924: <treelib.tree.Tree at 0x2d3455af128>,
 13325038116: <treelib.tree.Tree at 0x2d3455af4e0>,
 13854627199: <treelib.tree.Tree at 0x2d3455afda0>,
 13864739266: <treelib.tree.Tree at 0x2d3457d6438>,
 15954611837: <treelib.tree.Tree at 0x2d3457d6ba8>,
 15954688237: <treelib.tree.Tree at 0x2d3457ea7f0>,
 15963883482: <treelib.tree.Tree at 0x2d3457f6240>,
 15963885355: <treelib.tree.Tree at 0x2d3457f6668>,
 18554652702: <treelib.tree.Tree at 0x2d345803780>,
 18654692914: <treelib.tree.Tree at 0x2d345803e10>,
 18764569988: <treelib.tree.Tree at 0x2d345812ba8>,
 1590120464497: <treelib.tree.Tree at 0x2d34581f278>,
 1590120718170: <treelib.tree.Tree at 0x2d34581f390>,
 1590130640102: <treelib.tree.Tree at 0x2d34581fa58>,
 1590130817948: <treelib.tree.Tree at 0x2d34581fbe0>,
 1590140304209: <treelib.tree.Tree at 0x2d34582e198>,
 1590140304506: <treelib.tree.Tree at 0x2d345839c50>,
 1590140305107: <treelib.tree.Tree at 0x2d345839208>,
 1590140306678: <treelib.tree.Tree at 0x2d345846978>,
 

## 求unionTree

In [46]:
# 781924   13325038116
c_tree1 = ftctree_dict[781924]
c_tree2 = ftctree_dict[13325038116]
c_tree1.show()
c_tree2.show()

781924
├── 10
│   └── 101
├── 11
│   ├── 113
│   │   └── 1130
│   │       └── 11302
│   └── 115
│       └── 1153
├── 14
│   ├── 140
│   │   └── 1408
│   │       └── 14082
│   ├── 141
│   │   └── 1410
│   │       └── 14101
│   └── 144
├── 15
│   ├── 151
│   │   ├── 1511
│   │   └── 1512
│   │       └── 15120
│   └── 152
│       └── 1520
│           └── 15200
├── 23
│   └── 231
│       └── 2311
│           └── 23113
└── 34
    └── 340
        └── 3402
            └── 34023

13325038116
├── 10
│   ├── 101
│   │   └── 1015
│   ├── 102
│   │   └── 1020
│   ├── 103
│   └── 104
│       └── 1045
│           └── 10450
├── 14
│   └── 148
├── 15
│   └── 151
│       ├── 1511
│       └── 1512
│           └── 15120
├── 22
├── 23
│   └── 231
│       ├── 2311
│       └── 2313
│           └── 23132
├── 24
│   └── 244
└── 27
    └── 270
        └── 2700



In [50]:
# 求unionTree（利用迭代，temp_unionTree与新客户FTCTree进行union即可）
def union_tree(c_tree1, c_tree2):
    union_tree = Tree(tree=c_tree1, deep=True)
    for item in c_tree2.expand_tree('root'):
        if item == "root":
            continue
        if union_tree.contains(item):
            union_tree[item].data += c_tree2[item].data
        else:
            parent = c_tree2.parent(item).identifier
            union_tree.create_node(item, item, data = c_tree2[item].data, parent = parent)
    return union_tree

def inter_tree(c_tree1, c_tree2):
    u_tree = union_tree(c_tree1, c_tree2)
    inter_tree = Tree()
    inter_tree.create_node('ROOT','root')
    for item in u_tree.expand_tree('root'):
        if item == 'root':
            continue
        if c_tree1.contains(item) and c_tree2.contains(item):
            parent = u_tree.parent(item).identifier
            inter_tree.create_node(item, item, data = u_tree[item].data, parent = parent)
    return inter_tree

def cluster_union_tree(l_ftctree):
    if len(l_ftctree) == 0:
        return None
    elif len(l_ftctree) == 1:
        return l_ftctree[0]
    elif len(ftctree_dict) == 2:
        return union_tree(l_ftctree[0], l_ftctree[1])
    else:
#         cluster_union_tree = union_tree(ftctree_dict[list(ftctree_dict.keys())[0]], ftctree_dict[list(ftctree_dict.keys())[1]])
        temp_list = []
        is_odd = len(l_ftctree)%2
        if is_odd != 0: 
            temp_list.append(l_ftctree[0])
        for index, item in enumerate(l_ftctree):
            if is_odd != 0:
                if index % 2 == 0 and index != 0:
                    temp_list.append(union_tree(l_ftctree[index-1],l_ftctree[index]))
            else:
                if index % 2 == 1:
                    temp_list.append(union_tree(l_ftctree[index-1],l_ftctree[index]))
        c_union_tree = cluster_union_tree(temp_list)
        return c_union_tree

In [104]:
i_tree = inter_tree(c_tree1, c_tree2)
i_tree.show()
u_tree =  union_tree(c_tree1, c_tree2)
u_tree.show()
# pa = u_tree.parent(item).identifier

ROOT
├── 10
│   └── 101
├── 14
├── 15
│   └── 151
│       ├── 1511
│       └── 1512
│           └── 15120
└── 23
    └── 231
        └── 2311

781924
├── 10
│   ├── 101
│   │   └── 1015
│   ├── 102
│   │   └── 1020
│   ├── 103
│   └── 104
│       └── 1045
│           └── 10450
├── 11
│   ├── 113
│   │   └── 1130
│   │       └── 11302
│   └── 115
│       └── 1153
├── 14
│   ├── 140
│   │   └── 1408
│   │       └── 14082
│   ├── 141
│   │   └── 1410
│   │       └── 14101
│   ├── 144
│   └── 148
├── 15
│   ├── 151
│   │   ├── 1511
│   │   └── 1512
│   │       └── 15120
│   └── 152
│       └── 1520
│           └── 15200
├── 22
├── 23
│   └── 231
│       ├── 2311
│       │   └── 23113
│       └── 2313
│           └── 23132
├── 24
│   └── 244
├── 27
│   └── 270
│       └── 2700
└── 34
    └── 340
        └── 3402
            └── 34023



In [28]:
# 测试cluster_union_tree函数
t_ftc = {}
for index, item in enumerate(list(ftctree_dict.keys())):
            if index < 400:
                t_ftc[item] = ftctree_dict[item]
            else:
                break

test_u_tree = cluster_union_tree(list(t_ftc.values()))
print(test_u_tree.to_json(with_data=True))

{"781924": {"children": [{"10": {"children": [{"100": {"children": [{"1000": {"children": [{"10000": {"data": 19}}, {"10001": {"data": 4}}, {"10002": {"data": 9}}], "data": 76}}], "data": 148}}, {"101": {"children": [{"1010": {"children": [{"10100": {"data": 3}}, {"10103": {"data": 2}}, {"10109": {"data": 4}}], "data": 17}}, {"1011": {"children": [{"10110": {"data": 6}}, {"10111": {"data": 1}}, {"10112": {"data": 2}}, {"10113": {"data": 1}}, {"10114": {"data": 1}}, {"10116": {"data": 8}}, {"10119": {"data": 16}}], "data": 68}}, {"1013": {"children": [{"10130": {"data": 13}}, {"10131": {"data": 13}}, {"10132": {"data": 8}}, {"10133": {"data": 4}}, {"10135": {"data": 1}}, {"10136": {"data": 4}}, {"10137": {"data": 3}}], "data": 95}}, {"1014": {"children": [{"10140": {"data": 4}}, {"10141": {"data": 55}}], "data": 96}}, {"1015": {"children": [{"10150": {"data": 9}}, {"10151": {"data": 11}}, {"10152": {"data": 9}}], "data": 57}}], "data": 597}}, {"102": {"children": [{"1020": {"children": 

In [29]:
test_u_tree.show()

781924
├── 10
│   ├── 100
│   │   └── 1000
│   │       ├── 10000
│   │       ├── 10001
│   │       └── 10002
│   ├── 101
│   │   ├── 1010
│   │   │   ├── 10100
│   │   │   ├── 10103
│   │   │   └── 10109
│   │   ├── 1011
│   │   │   ├── 10110
│   │   │   ├── 10111
│   │   │   ├── 10112
│   │   │   ├── 10113
│   │   │   ├── 10114
│   │   │   ├── 10116
│   │   │   └── 10119
│   │   ├── 1013
│   │   │   ├── 10130
│   │   │   ├── 10131
│   │   │   ├── 10132
│   │   │   ├── 10133
│   │   │   ├── 10135
│   │   │   ├── 10136
│   │   │   └── 10137
│   │   ├── 1014
│   │   │   ├── 10140
│   │   │   └── 10141
│   │   └── 1015
│   │       ├── 10150
│   │       ├── 10151
│   │       └── 10152
│   ├── 102
│   │   └── 1020
│   │       ├── 10200
│   │       ├── 10201
│   │       └── 10203
│   ├── 103
│   │   ├── 1030
│   │   │   └── 10300
│   │   └── 1031
│   │       └── 10310
│   ├── 104
│   │   ├── 1040
│   │   │   └── 10401
│   │   ├── 1042
│   │   │   └── 10420
│   │   ├── 1043
│   │   │   └── 10

In [30]:
test_u_tree.children('root')

[Node(tag=15, identifier=15, data=2901),
 Node(tag=11, identifier=11, data=1073),
 Node(tag=14, identifier=14, data=3551),
 Node(tag=34, identifier=34, data=147),
 Node(tag=10, identifier=10, data=1385),
 Node(tag=23, identifier=23, data=1125),
 Node(tag=22, identifier=22, data=3099),
 Node(tag=24, identifier=24, data=511),
 Node(tag=27, identifier=27, data=1872),
 Node(tag=25, identifier=25, data=575),
 Node(tag=30, identifier=30, data=1295),
 Node(tag=20, identifier=20, data=119),
 Node(tag=21, identifier=21, data=168),
 Node(tag=31, identifier=31, data=78),
 Node(tag=33, identifier=33, data=31),
 Node(tag=32, identifier=32, data=89),
 Node(tag=36, identifier=36, data=26),
 Node(tag=40, identifier=40, data=29)]

In [31]:
# 求AvgFreq(utree)和MaxFreq(utree)
def AvgFreq(utree):
    freq_list = []
    for item in utree.expand_tree('root'):
        if item == "root":
            continue
        freq_list.append(utree[item].data)
    return mean(freq_list)

avg = AvgFreq(test_u_tree)
avg

45.90189125295508

In [32]:

def MaxFreq(utree):
    freq_list = []
    for item in utree.expand_tree('root'):
        if item == "root":
            continue        
        freq_list.append(utree[item].data)
    m = max(freq_list)
    return m

m = MaxFreq(test_u_tree)
m

3551

In [33]:
# 实现update(utree,freq)函数
def update(utree, freq):
    updt_tree = Tree(tree=utree,deep=True)
    for item in utree.expand_tree('root'):
        if item == 'root':
            continue
        if updt_tree.contains(item) and utree[item].data < freq:
            updt_tree.remove_node(item)
    return updt_tree

updt_tree = update(test_u_tree, 3)
updt_tree.show()

781924
├── 10
│   ├── 100
│   │   └── 1000
│   │       ├── 10000
│   │       ├── 10001
│   │       └── 10002
│   ├── 101
│   │   ├── 1010
│   │   │   ├── 10100
│   │   │   └── 10109
│   │   ├── 1011
│   │   │   ├── 10110
│   │   │   ├── 10116
│   │   │   └── 10119
│   │   ├── 1013
│   │   │   ├── 10130
│   │   │   ├── 10131
│   │   │   ├── 10132
│   │   │   ├── 10133
│   │   │   ├── 10136
│   │   │   └── 10137
│   │   ├── 1014
│   │   │   ├── 10140
│   │   │   └── 10141
│   │   └── 1015
│   │       ├── 10150
│   │       ├── 10151
│   │       └── 10152
│   ├── 102
│   │   └── 1020
│   │       └── 10200
│   ├── 103
│   │   ├── 1030
│   │   │   └── 10300
│   │   └── 1031
│   │       └── 10310
│   ├── 104
│   │   ├── 1040
│   │   │   └── 10401
│   │   └── 1043
│   └── 105
├── 11
│   ├── 110
│   │   ├── 1100
│   │   ├── 1101
│   │   ├── 1102
│   │   ├── 1104
│   │   └── 1105
│   │       ├── 11054
│   │       ├── 11056
│   │       └── 11057
│   ├── 111
│   │   ├── 1110
│   │   ├── 1111
│   │

In [34]:
itree = Tree()
itree.create_node(tag="INTER", identifier="root",data= 8)
itree.create_node(15, 15, data = 5, parent="root")
itree.create_node(12, 12, data = 3, parent="root")
itree.create_node(151, 151, data = 3, parent=15)
itree.create_node(152, 152, data = 2, parent=15)
itree.create_node(121, 121, data = 3, parent=12)
itree.create_node(1211, 1211, data = 3, parent=121)
itree.show()

INTER
├── 12
│   └── 121
│       └── 1211
└── 15
    ├── 151
    └── 152



## 实现Dist(FTCTree,utree)函数

In [35]:
# 求树某深度的所有节点
def level_node(tree,n):
    lev_n_list = []
    for item in tree.expand_tree('root'):
        if tree.level(item) == n:
            lev_n_list.append(item)
    return lev_n_list


# 求每一层的每个节点的Sim（v,utree）
# sim_node_dic = {lev1:{identifi1: sim}}
def sim_node(ftctree,utree):
    sim_node_dic = {}
    for dp in range(1,ftctree.depth()+1):
        sim_node_dic[dp] = {}
        ftc_dp_list = level_node(ftctree, dp)
        if dp == 1:
            union_dp_list = level_node(utree, dp)
            union_dp_freq_sum = 0
            for node_inden in union_dp_list:
                union_dp_freq_sum += utree[node_inden].data
            for node_inden in ftc_dp_list:
                sim_node_dic[dp][node_inden] = ftctree[node_inden].data/union_dp_freq_sum
        else:
            for node_inden in ftc_dp_list:
                sim_node_dic[dp][node_inden] = ftctree[node_inden].data/utree.parent(node_inden).data
    return sim_node_dic


    # 求每一层的Sim（Vl,tree）
def sim_lev(ftctree, sim_node_dic):
    sim_lev_dic = {}
    for lev in sim_node_dic:
        if lev == 1:
            sim_lev_dic[lev] = sum(list(sim_node_dic[lev].values()))
        else:
            l_par = []
            for index in sim_node_dic[lev]:
                l_par.append(ftctree.parent(index))
            l_par = list(set(l_par))
            par_num = len(l_par)
            sim_lev_dic[lev] = sum(list(sim_node_dic[lev].values()))/par_num
    return sim_lev_dic


def Dist(ftctree,utree):
    sim_node_dic = sim_node(ftctree,utree)
    sim_lev_dic = sim_lev(ftctree,sim_node_dic)
    dist = 1
    lev_sum = sum(list(sim_lev_dic.keys()))
    for lev in  sim_lev_dic:
        dist -= sim_lev_dic[lev]*lev/lev_sum
    sim_node_dic
    sim_lev_dic
    return dist


dist = Dist(itree,itree)

dist

0.0

In [36]:
utree = cluster_union_tree(list(ftctree_dict.values()))
utree
dist = Dist(inter_tree(cluster[index],utree),union_tree(cluster[index],utree))

## 实现GetCT（cluster = ftctree_dict）

In [103]:
# 实现GetCT（cluster = ftctree_dict）
def GetCT(cluster):
    utree = cluster_union_tree(list(cluster.values()))
    freq = 1
    mindist = float('Inf')
    freqStep = AvgFreq(utree)
    freqEnd = MaxFreq(utree)
#     num_sum_nodes = 0
#     for item in list(cluster.values()):
#         num_sum_nodes += len(item.all_nodes())
#     num_avg_nodes = num_sum_nodes/len(list(cluster.values()))
    while freq <= freqEnd:
        utree = update(utree, freq)
        dist = 0
        for index in cluster:
            dist += Dist(inter_tree(cluster[index],utree),union_tree(cluster[index],utree))
        if dist < mindist:
            mindist = dist
            ct = utree
#             if len(ct.all_nodes()) <= num_avg_nodes:
#                 break
        freq = freq + freqStep
    return ct
ct = GetCT(ftctree_dict)
ct

In [102]:
ct.show()

781924
├── 10
│   └── 101
├── 11
│   └── 115
├── 14
│   ├── 140
│   ├── 142
│   ├── 147
│   └── 148
├── 15
│   ├── 151
│   │   └── 1511
│   └── 152
├── 22
│   ├── 220
│   └── 221
│       └── 2210
├── 23
│   └── 231
├── 24
├── 25
│   └── 251
├── 27
│   ├── 270
│   │   └── 2700
│   └── 274
└── 30
    └── 303
        └── 3038



## 实现簇的分裂算法

In [71]:
# KMeans K=2

# 1 初始化质心（随机选取两个用户点随机选择N对值，选取一对距离最大的质心。）
#   缺点：选择的用户树可能数据量很小。不足以代表，可添加约束条件，选择的种子树结点个数不小于平均结点个数。
#   初始化簇，开始时簇中包含两质心
# 2 计算每个用户点到质心的距离，比较后加入距离较短的簇中
# 3 更新质心，判断两个簇是否发生变化，变化则执行2，不变则结束算法

# 计算簇中所有树的平均节点个数
def avg_nodes_num(ftctree_dict):
    sum_nodes_num = 0
    for index in ftctree_dict:
        sum_nodes_num += len(ftctree_dict[index].all_nodes())
    return sum_nodes_num/len(ftctree_dict)

# 初始化质心
def calculate_initial_centroids_first_run(ftctree_dict):
    avg_n_sum = avg_nodes_num(ftctree_dict)
    init_centroids_list = []
    init_dist_list = []
    i = 0
    while i < 10:
        temp_c = random.sample(list(ftctree_dict.keys()), 2)
        if len(ftctree_dict[temp_c[0]]) >= avg_n_sum and len(ftctree_dict[temp_c[1]]) >= avg_n_sum:
            init_centroids_list.append(temp_c)
            i += 1
    for item in init_centroids_list:
        init_dist_list.append(Dist(inter_tree(ftctree_dict[item[0]], ftctree_dict[item[1]]), union_tree(ftctree_dict[item[0]], ftctree_dict[item[1]])))
    max_dist_index = init_dist_list.index(max(init_dist_list))
    return {init_centroids_list[max_dist_index][0]:ftctree_dict[init_centroids_list[max_dist_index][0]], init_centroids_list[max_dist_index][1]:ftctree_dict[init_centroids_list[max_dist_index][1]]}

centroids = calculate_initial_centroids_first_run(ftctree_dict)
centroids
# a = init_centroids[list(init_centroids.keys())[0]]
# b = init_centroids[list(init_centroids.keys())[1]]
# Dist(inter_tree(a,b), union_tree(a,b))

{1591012978559: <treelib.tree.Tree at 0x2d345b8a438>,
 2900001550246: <treelib.tree.Tree at 0x2d34639a748>}

In [70]:
# # 初始化簇，开始时簇中包含两质心
# def init_cluster(centroids):
#     cluster = {}
#     ct1 = list(centroids.keys())[0]
#     ct2 = list(centroids.keys())[1]
#     cluster[ct1] = [ct1]
#     cluster[ct2] = [ct2]
#     return cluster

In [74]:
# 计算每个用户点到质心的距离，比较后加入距离较短的簇中(更新簇)
def calculate_cluster(ftctree_dict, centroids):
    cluster = {}
    ct1 = list(centroids.keys())[0]
    ct2 = list(centroids.keys())[1]
    cluster[ct1] = []
    cluster[ct2] = []
    ct1_tree = centroids[ct1]
    ct2_tree = centroids[ct2]
    for index in ftctree_dict:
        dist1 = Dist(inter_tree(ftctree_dict[index],ct1_tree), union_tree(ftctree_dict[index],ct1_tree))
        dist2 = Dist(inter_tree(ftctree_dict[index],ct2_tree), union_tree(ftctree_dict[index],ct2_tree))
        if dist1 > dist2:
            cluster[ct2].append(index)
        else:
            cluster[ct1].append(index)
    return cluster

cluster = calculate_cluster(ftctree_dict, centroids)
cluster

{1591012978559: [781924,
  13325038116,
  13854627199,
  13864739266,
  15954611837,
  15954688237,
  15963883482,
  15963885355,
  18764569988,
  1590130640102,
  1590140304209,
  1590140304506,
  1590140305107,
  1590140306678,
  1590140307415,
  1590140307767,
  1590140307859,
  1590140308030,
  1590140308214,
  1590141216259,
  1590142129763,
  1590142148962,
  1590142149112,
  1590142150507,
  1590142151689,
  1590142176354,
  1590142176521,
  1590142179621,
  1590142191722,
  1590142201148,
  1590142201162,
  1590142201513,
  1590142202190,
  1590142203104,
  1590142203401,
  1590142204569,
  1590142205993,
  1590142206747,
  1590142213189,
  1590142213356,
  1590142213981,
  1590142215619,
  1590142230179,
  1590142456470,
  1590142477895,
  1590142516068,
  1590142518307,
  1590142520300,
  1590142751995,
  1590150321449,
  1590151076256,
  1590151321240,
  1590151363066,
  1590151432557,
  1590151464053,
  1590151500423,
  1590151825175,
  1590151829753,
  1590151832586,
  159

In [77]:
# 更新质心
def update_centroids(ftctree_dict, cluster):
    ct1 = list(cluster.keys())[0]
    ct2 = list(cluster.keys())[1]
    cluster1_dict = {}
    cluster2_dict = {}
    for item in cluster[ct1]:
        cluster1_dict[item] = ftctree_dict[item]
    for item in cluster[ct2]:
        cluster2_dict[item] = ftctree_dict[item]
    centroids = {}
    centroids['ct1'] = GetCT(cluster1_dict)
    centroids['ct2'] = GetCT(cluster2_dict)
    return centroids
centroids = update_centroids(ftctree_dict, cluster)
centroids

{'ct1': <treelib.tree.Tree at 0x2d342dc8080>,
 'ct2': <treelib.tree.Tree at 0x2d34a1aa0f0>}

In [94]:
centroids['ct1'].show()
centroids['ct2'].show()

781924
├── 14
├── 15
└── 22

18554652702
└── 14



In [79]:
# 判断簇是否发生变化
def is_change(last_cluster, cluster):
    equ_n = 0
    for index in cluster:
        for index2 in last_cluster:
            if operator.eq(cluster[index], last_cluster[index2]):
                equ_n += 1
    if equ_n == 2:
        return False
    return True

In [91]:
# 实现Kmeans
def Kmeans(cluster_dict):
    centroids = calculate_initial_centroids_first_run(cluster_dict)
    cluster = calculate_cluster(cluster_dict, centroids)
    last_cluster = deepcopy(cluster)
    isChange = True
    while isChange:
        centroids = update_centroids(cluster_dict, cluster)
        cluster = calculate_cluster(cluster_dict, centroids)
        isChange = is_change(last_cluster, cluster)
        last_cluster = cluster
    result = {}
    result['cluster'] = cluster
    result['centroids'] = centroids
    return result

In [106]:
# 实现求bic函数，对于分裂前后的簇，分别求bic
def calculate_sigma(cluster_dict, ct_tree, clu_num):
    dist = 0
    for index in cluster_dict:
        dist += Dist(inter_tree(cluster_dict[index],ct_tree),union_tree(cluster_dict[index],ct_tree))**2
    sigma = (1/(len(cluster_dict) - clu_num))*dist
    return sigma

def plu_num(cluster_dict):
    l_p = []
    for index in cluster_dict:
        l_leaves = cluster_dict[index].leaves()
        for item in l_leaves:
            if cluster_dict[index].depth(item) == 4:
                l_p.append(item.identifier)
    return len(list(set(l_p)))
            

def calculate_likelihood_before(cluster_dict, ct_tree, clu_num):
    c_num = len(cluster_dict)
    likhood = c_num*math.log(c_num) - c_num*math.log(c_num) - (c_num/2)*math.log(2*math.pi)\
                - (c_num*plu_num(cluster_dict)/2)*math.log(calculate_sigma(cluster_dict, ct_tree, clu_num)) - (c_num - clu_num)/2
    return likhood

def calculate_likelihood_after(cluster_dict,cluster_dict1,ct1_tree,cluster_dict2,ct2_tree,clu_num):
    c_num1 = len(cluster_dict1)
    likhood = c_num1*math.log(c_num1) - c_num1*math.log(len(cluster_dict)) - (c_num1/2)*math.log(2*math.pi)\
                - (c_num1*plu_num(cluster_dict1)/2)*math.log(calculate_sigma(cluster_dict1, ct1_tree, clu_num)) - (c_num1 - clu_num)/2
    c_num2 = len(cluster_dict2)
    likhood += c_num2*math.log(c_num2) - c_num2*math.log(len(cluster_dict)) - (c_num2/2)*math.log(2*math.pi)\
                - (c_num2*plu_num(cluster_dict2)/2)*math.log(calculate_sigma(cluster_dict2, ct2_tree, clu_num)) - (c_num2 - clu_num)/2
    return likhood

def BIC_before(cluster_dict, ct_tree, clu_num):
    bic = calculate_likelihood_before(cluster_dict, ct_tree, clu_num) - (clu_num*(plu_num(cluster_dict)+1)/2)*math.log(clu_num)
    return bic

def BIC_after(cluster_dict,cluster_dict1,ct1_tree,cluster_dict2,ct2_tree,clu_num):
    bic = calculate_likelihood_after(cluster_dict,cluster_dict1,ct1_tree,cluster_dict2,ct2_tree,clu_num) - (clu_num*(plu_num(cluster_dict)+1)/2)*math.log(clu_num)
    return bic

In [107]:
def FTCAlgorithm(ftctree_dict):
    final_cluster = []
    final_centroids = []
    final_result = {}
    ct_tree = GetCT(ftctree_dict)
    try:
        result = Kmeans(ftctree_dict)
    except:
        final_cluster .append(list(ftctree_dict.keys()))
        final_centroids.append(ct_tree)
        final_result['cluster'] = final_cluster
        final_result['centroids'] = final_centroids
        return final_result
    cluster = result['cluster']
    ct1 = list(cluster.keys())[0]
    ct2 = list(cluster.keys())[1]
    cluster1_dict = {}
    cluster2_dict = {}
    ct1_tree = result['centroids'][ct1]
    ct2_tree = result['centroids'][ct2]
    for item in cluster[ct1]:
        cluster1_dict[item] = ftctree_dict[item]
    for item in cluster[ct2]:
        cluster2_dict[item] = ftctree_dict[item]
    before_bic = BIC_before(ftctree_dict,ct_tree,1)
    after_bic = BIC_after(ftctree_dict,cluster1_dict,ct1_tree,cluster2_dict,ct2_tree,2)
    if before_bic < after_bic:
        temp_result1 = FTCAlgorithm(cluster1_dict)
        temp_result2 = FTCAlgorithm(cluster2_dict)
        for item in temp_result1['cluster']:
            final_cluster.append(item)
        for item in temp_result1['centroids']:
            final_centroids.append(item)
        for item in temp_result2['cluster']:
            final_cluster.append(item)
        for item in temp_result2['centroids']:
            final_centroids.append(item)
        final_result['cluster'] = final_cluster
        final_result['centroids'] = final_centroids
        return final_result
    else:
        final_cluster.append(list(ftctree_dict.keys()))
        final_centroids.append(ct_tree)
        final_result['cluster'] = final_cluster
        final_result['centroids'] = final_centroids
        return final_result
final_result = FTCAlgorithm(ftctree_dict)
final_result

{'cluster': [[781924,
   13854627199,
   15954611837,
   1590120718170,
   1590142152440,
   1590142176354,
   1590142179621,
   1590142201513,
   1590142210195,
   1590142215619,
   1590142230179,
   1590142240482,
   1590142516600,
   1590142517836,
   1590142751995,
   1590151076256,
   1590151428581,
   1590151457185,
   1590151459721,
   1590151467962,
   1590151682891,
   1590151977348,
   1591013227632,
   1591013877134,
   1591014130276,
   1591014133086,
   1591014214433,
   1591014255948,
   1591014523610,
   1591014637324,
   1591014943968,
   1591015027858,
   1591015088262,
   1591015091286,
   1591015184407,
   1591015476700,
   1591015480455,
   1591015480745,
   1591015501716,
   1591016159916,
   1591016320026,
   1591016350504,
   1591016439575,
   1591016443121,
   1591016494765,
   1591040155274,
   1591040203906,
   1591040497398,
   1592015058774,
   1593141148827,
   1594140121125,
   1595150263683,
   1595150403423,
   1595150738747,
   1595150738976,
   1595150

In [113]:
# 计算轮廓系数
def silhouette_score(ftctree_dict, cluster):
    sc = []
    # 遍历所有簇
    for index, item in enumerate(cluster):
        # 遍历簇中所有用户点
        for item2 in item:
            a = 0
            b_list = []
            b = 0
            # 对于每一个用户点，遍历簇中其他所有用户点
            for item3 in item:
                if item2 == item3:
                    continue
                t1 = ftctree_dict[item2]
                t2 = ftctree_dict[item3]
                a += Dist(inter_tree(t1,t2),union_tree(t1, t2))
            # 可能该簇中只有当前用户点一个点，此时a=0
            if a != 0:
                a = a/(len(item) - 1)

            # 遍历当前簇之外的其他簇
            for index4, item4 in enumerate(cluster):
                if index == index4:
                    continue
                # 遍历其他簇中所有用户点
                for item5 in item4:
                    t1 = ftctree_dict[item2]
                    t2 = ftctree_dict[item5]
                    b += Dist(inter_tree(t1, t2), union_tree(t1, t2))
                b = b/len(item4)
                b_list.append(b)
            b = min(b_list)
            sc.append((b-a)/max([a, b]))
    return mean(sc)


# 计算紧密性
def compactness(ftctree_dict, centroids, cluster):
    # 遍历所有簇
    for index, item in enumerate(cluster):
        # 每个簇中所有点到质心的距离列表
        cpi = []
        # 每个簇的a_cpi = avg(cpi)列表
        apk = []
        # 遍历当前簇中所有点
        t2 = centroids[index]
        for item2 in item:
            t1 = ftctree_dict[item2]
            cpi.append(Dist(inter_tree(t1, t2), union_tree(t1, t2)))
        apk.append(mean(cpi))
    return mean(apk)


In [114]:
final_result = FTCAlgorithm(ftctree_dict)
print('-----------------------')
print('final_result')
print(final_result)
print('-----------------------')

print('-----------------------')
print('sc')
print(silhouette_score(ftctree_dict, final_result['cluster']))
print('-----------------------')

print('-----------------------')
print('cp')
print(compactness(ftctree_dict, final_result['centroids'], final_result['cluster']))
print('-----------------------')

for item in final_result['centroids']:
    item.show()

-----------------------
final_result
{'cluster': [[18764569988, 1590142149112, 1590142201513, 1590142202190, 1590142240482, 1590151427355, 1590151457185, 1590151500423, 1590151829753, 1591012978559, 1591015034924, 1591015091286, 1591015112424, 1591015454142, 1591015456993, 1591015598785, 1591016162466, 1591016382857, 1591020695158, 1593140971419, 1594140121125, 1595140612422, 1595150738747, 1595150997786, 1595151355660, 1595151375125, 1595151630774, 1595151644528, 1598140055923, 2900000050419, 2900000060012, 2900000072701, 2900000138223, 2900000192492, 2900000254503, 2900000304376, 2900000348561, 2900000384187, 2900000385627, 2900000390256, 2900000516892, 2900000574274, 2900000598973, 2900000623040, 2900000776104, 2900000780040, 2900000780835, 2900000855502, 2900000856585, 2900001053938, 2900001137843, 2900001222099, 2900001240246, 2900001270014, 2900001582933, 2900001645768, 2900002944495, 2900003116341], [781924, 13325038116, 13854627199, 13864739266, 15954611837, 15954688237, 159638